In [ ]:
%load_ext autoreload

In [ ]:
# Import necessary libraries
%autoreload
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

In [ ]:
energy_df = pd.read_csv('../data/energy_import_export.csv')
solar_df = pd.read_csv('../data/solar_self_consumption_main_building.csv')
met_df = pd.read_csv('../data/met_data.csv')
